# Labeling Cell Images
## Imports

In [1]:
import cv2
import numpy as np
import os

from config import *
from CellProcessor import read_image, process_image, get_bboxes, get_label_yolo

/Users/sapplab/Desktop/CellDataProcessing/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Make dir for labels

In [2]:
LABELED_IMAGES_DIR_PATH = IMAGES_PATH + "Labeles_phase/"
IMAGES_MASKS_DIR_PATH = IMAGES_PATH + "Masks_phase/"
IMAGES_PHASE_PATH = IMAGES_PATH + "TSV_Labeled_phase/"
try:
    os.mkdir(LABELED_IMAGES_DIR_PATH)
except OSError:
    pass

try:
    os.mkdir(IMAGES_MASKS_DIR_PATH)
except OSError:
    pass

## For every iamge gen label

In [19]:
images = os.listdir(GREEN_PATH)
len_images = len(images)
curr_image_num = 1

for image in images:
    print(str(curr_image_num) + "/" + str(len_images))
    curr_image_num+=1
    green_image_path = GREEN_PATH + image
    base_image_path = PHASE_PATH + image
    img, img_base = read_image(green_image_path, base_image_path)

    img_base_h, img_base_w = img_base.shape[:2]

    CROP_WIDTH = 128
    CROP_HEIGHT = 128
    CROP_WIDTH_OVERLAP = 64
    CROP_HEIGHT_OVERLAP = 52
    CROP_WIDTH_SHIFT = CROP_WIDTH - CROP_WIDTH_OVERLAP
    CROP_HEIGHT_SHIFT = CROP_HEIGHT - CROP_HEIGHT_OVERLAP
    num_width_windows = (img_base_w - CROP_WIDTH)/CROP_WIDTH_SHIFT
    num_height_windows = (img_base_h - CROP_HEIGHT)/CROP_HEIGHT_SHIFT

    for width_window in range(int(num_width_windows)):
        for height_window in range(int(num_height_windows)):
            curr_topleft_x = width_window * CROP_WIDTH_SHIFT
            curr_topleft_y = height_window * CROP_HEIGHT_SHIFT
            img_crop = img[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]
            img_base_crop = img_base[curr_topleft_y:curr_topleft_y + CROP_HEIGHT, curr_topleft_x:curr_topleft_x + CROP_WIDTH]

            img_dilation = process_image(img_crop, 
                                contrast, 
                                brightness, 
                                threshold, 
                                i_erode, 
                                i_dialate, 
                                plot = False)

            bboxes = get_bboxes(img_dilation)
            label_boxes = []
            if len(bboxes) > 0:
                img_mask_file = IMAGES_MASKS_DIR_PATH + image.split(".")[0] + "_" + str(width_window) + "_" + str(height_window) + ".png"
                img_file = IMAGES_PHASE_PATH + image.split(".")[0] + "_" + str(width_window) + "_" + str(height_window) + ".png"
                cv2.imwrite(img_mask_file, img_dilation)
                cv2.imwrite(img_file, img_base_crop)
                label_file = LABELED_IMAGES_DIR_PATH + "/" + image.split(".")[0] + "_" + str(width_window) + "_" + str(height_window) + ".txt"
                for start_pt, end_pt in bboxes:
                    img_h, img_w = img_dilation.shape[:2]
                    x, y, w, h = get_label_yolo(start_pt, end_pt, img_w, img_h)
                    label_box = "0 " + str(x) + " " + str(y) + " " + str(w) + " " + str(h) + "\n"
                    label_boxes.append(label_box)

                with open(label_file, "w") as ann:
                    for lb in label_boxes:
                        ann.write(lb)

1/312
1040
12.0
20.0
2/312
1040
12.0
20.0
3/312
1040
12.0
20.0
4/312
1040
12.0
20.0
5/312
1040
12.0
20.0
6/312
1040
12.0
20.0
7/312
1040
12.0
20.0
8/312
1040
12.0
20.0
9/312
1040
12.0
20.0
10/312
1040
12.0
20.0
11/312
1040
12.0
20.0
12/312
1040
12.0
20.0
13/312
1040
12.0
20.0
14/312
1040
12.0
20.0
15/312
1040
12.0
20.0
16/312
1040
12.0
20.0
17/312
1040
12.0
20.0
18/312
1040
12.0
20.0
19/312
1040
12.0
20.0
20/312
1040
12.0
20.0
21/312
1040
12.0
20.0
22/312
1040
12.0
20.0
23/312
1040
12.0
20.0
24/312
1040
12.0
20.0
25/312
1040
12.0
20.0
26/312
1040
12.0
20.0
27/312
1040
12.0
20.0
28/312
1040
12.0
20.0
29/312
1040
12.0
20.0
30/312
1040
12.0
20.0
31/312
1040
12.0
20.0
32/312
1040
12.0
20.0
33/312
1040
12.0
20.0
34/312
1040
12.0
20.0
35/312
1040
12.0
20.0
36/312
1040
12.0
20.0
37/312
1040
12.0
20.0
38/312
1040
12.0
20.0
39/312
1040
12.0
20.0
40/312
1040
12.0
20.0
41/312
1040
12.0
20.0
42/312
1040
12.0
20.0
43/312
1040
12.0
20.0
44/312
1040
12.0
20.0
45/312
1040
12.0
20.0
46/312
1040
12.0
20

In [ ]:
#----------------------------------OLD----------------------------------
images = os.listdir(GREEN_PATH)
len_images = len(images)
curr_image_num = 1

for image in images:
    print(str(curr_image_num) + "/" + str(len_images))
    curr_image_num+=1
    green_image_path = GREEN_PATH + image
    base_image_path = PHASE_PATH + image
    img, img_base = read_image(green_image_path, base_image_path)
    # initial preporcess and contours
    img_dilation = process_image(img, 
                                contrast, 
                                brightness, 
                                threshold, 
                                i_erode, 
                                i_dialate, 
                                plot = False)
    
    img_mask_file = IMAGES_MASKS_DIR_PATH + image
    cv2.imwrite(img_mask_file, img_dilation)
    
    bboxes = get_bboxes(img_dilation)
    label_boxes = []
    for start_pt, end_pt in bboxes:
        img_h, img_w = img_dilation.shape[:2]
        x, y, w, h = get_label_yolo(start_pt, end_pt, img_w, img_h)
        label_file = LABELED_IMAGES_DIR_PATH + "/" + image.split(".")[0] + ".txt"
        label_box = "0 " + str(x) + " " + str(y) + " " + str(w) + " " + str(h) + "\n"
        label_boxes.append(label_box)
    with open(label_file, "w") as ann:
        for lb in label_boxes:
            ann.write(lb)